## Methods

- RK45:
  - Explicit Runge-Kutta method of order 5(4). The error is controlled assuming accuracy of the fourth- order method, but steps are taken using the fifth-order accurate formula (local extrapolation is done). A quartic interpolation polynomial is used for the dense output. Can be applied in the complex domain.

- RK23:
  - Explicit Runge-Kutta method of order 3(2). The error is controlled assuming accuracy of the second-order method, but steps are taken using the third-order accurate formula (local extrapolation is done). A cubic Hermite polynomial is used for the dense output. Can be applied in the complex domain.
    
- DOP853:
  - Explicit Runge-Kutta method of order 8. Python implementation of the "DOP853" algorithm originally written in Fortran. A 7-th order interpolation polynomial accurate to 7-th order is used for the dense output. Can be applied in the complex domain.
    
- Radau:
  - Implicit Runge-Kutta method of the Radau IIA family of order 5. The error is controlled with a third-order accurate embedded formula. A cubic polynomial which satisfies the collocation conditions is used for the dense output.
    
- BDF:
  - Implicit multi-step variable-order (1 to 5) method based on a backward differentiation formula for the derivative approximation. A quasi-constant step scheme is used and accuracy is enhanced using the NDF modification. Can be applied in the complex domain.
    
- LSODA:
  - Adams/BDF method with automatic stiffness detection and switching. This is a wrapper of the Fortran solver from ODEPACK.

Explicit Runge-Kutta methods ('RK23', 'RK45', 'DOP853') should be used for non-stiff problems and implicit methods ('Radau', 'BDF') for stiff problems.

Among Runge-Kutta methods, 'DOP853' is recommended for solving with high precision (low values of `rtol` and `atol`).

If not sure, first try to run 'RK45'.

If it makes unusually many iterations, diverges, or fails, your problem is likely to be stiff and you should use 'Radau' or 'BDF'.

'LSODA' can also be a good universal choice, but it might be somewhat less convenient to work with as it wraps old Fortran code.

# scipy.solve_ivp

Solve an initial value problem for a system of ODEs.

This function numerically integrates a system of ordinary differential equations given an initial value:

$$\frac{dy}{dt}=f(t,y),$$
$$y(t_0) =y_0$$

Here $t$ is a one-dimensional independent variable (time), $y(t)$ is an n-dimensional vector-valued function (state), and an n-dimensional vector-valued function $f(t, y)$ determines the differential equations.

The goal is to find $y(t)$ approximately satisfying the differential equations, given an initial value $y(t_0)$=$y_0$.

Some of the solvers support integration in the complex domain, but note that for stiff ODE solvers, the right-hand side must be complex-differentiable (satisfy Cauchy-Riemann equations). To solve a problem in the complex domain, pass $y_0$ with a complex data type. Another option always available is to rewrite your problem for real and imaginary parts separately.

Parameters
----------
1. `fun` : callable
    Right-hand side of the system.
    The calling signature is `fun(t,y)`.
    Here `t` is a scalar, and there are two options for the ndarray `y`:
      - It can either have shape (n,); then `fun` must return array_like with shape (n,).
      - Alternatively it can have shape (n, k); then `fun` must return an array_like with shape (n, k), i.e. each column corresponds to a single column in `y`. The choice between the two options is determined by `vectorized` argument (see below). The vectorized implementation allows a faster approximation of the Jacobian by finite differences (required for stiff solvers).
2. `t_span` : 2-tuple of floats

    Interval of integration (`t0`, `tf`).
    The solver starts with `t=t0` and integrates until it reaches `t=tf`.
3. `y0` : array_like, shape (n,)

    Initial state.
    For problems in the complex domain, pass `y0` with a complex data type (even if the initial value is purely real).
4. `method` : `str` or `OdeSolver`, optional
    * 'RK45'
    * 'RK23'
    * 'DOP853'
    * 'Radau'
    * 'BDF'
    * 'LSODA'
5. `t_eval` : `array_like` or `None`, optional

    Times at which to store the computed solution, must be sorted and lie within `t_span`.
    If None (default), use points selected by the solver.
6. `dense_output` : `bool`, optional

    Whether to compute a continuous solution. Default is `False`.
8. `vectorized` : `bool`, optional

    Whether `fun` is implemented in a vectorized fashion.
    Default is False.
9. `args` : `tuple`, optional

    Additional arguments to pass to the user-defined functions.
    If given, the additional arguments are passed to all user- defined functions.
    So if, for example, `fun` has the signature `fun(t, y, a, b, c)`, then `jac` (if given) and any event functions must have the same signature, and `args` must be a tuple of length 3.
10. `options`

    Options passed to a chosen solver.
    All options available for already implemented solvers are listed below.
11. `first_step` : `float` or `None`, optional

    Initial step size. Default is `None` which means that the algorithm should choose.
12. `max_step` : `float`, optional

    Maximum allowed step size.
13. `rtol`, `atol` : `float` or `array_like`, optional

    Relative and absolute tolerances.
    The solver keeps the local error estimates less than `atol + rtol * abs(y)`.
    Here `rtol` controls a relative accuracy (number of correct digits).
    But if a component of `y` is approximately below `atol`, the error only needs to fall within the same `atol` threshold, and the number of correct digits is not guaranteed.
    If components of y have different scales, it might be beneficial to set different `atol` values for different components by passing array_like with shape (n,) for `atol`.
    Default values are 1e-3 for `rtol` and 1e-6 for `atol`.
14. `jac` : `array_like`, `sparse_matrix`, `callable` or `None`, optional

    Jacobian matrix of the right-hand side of the system with respect to y, required by the 'Radau', 'BDF' and 'LSODA' method.
    The Jacobian matrix has shape (n, n) and its element (i, j) is equal to `df_i / dy_j`.
    There are three ways to define the Jacobian:
    - If array_like or sparse_matrix, the Jacobian is assumed to be constant.
        Not supported by 'LSODA'.
    - If callable, the Jacobian is assumed to depend on both t and y; it will be called as `jac(t, y)` as necessary.
        For 'Radau' and 'BDF' methods, the return value might be a sparse matrix.
    - If None (default), the Jacobian will be approximated by finite differences.
    It is generally recommended to provide the Jacobian rather than relying on a finite-difference approximation.
15. `jac_sparsity` : `array_like`, `sparse_matrix` or `None`, optional

    Defines a sparsity structure of the Jacobian matrix for a finite-difference approximation.
    Its shape must be (n, n).
    This argument is ignored if `jac` is not `None`.
    If the Jacobian has only few non-zero elements in *each* row, providing the sparsity structure will greatly speed up the computations.
    A zero entry means that a corresponding element in the Jacobian is always zero.
    If None (default), the Jacobian is assumed to be dense.
    Not supported by 'LSODA', see `lband` and `uband` instead.
16. `lband`, `uband` : `int` or `None`, optional

    Parameters defining the bandwidth of the Jacobian for the 'LSODA' method, i.e., `jac[i, j] != 0 only for i - lband <= j <= i + uband`.
    Default is None.
    Setting these requires your `jac` routine to return the Jacobian in the packed format: the returned array must have `n` columns and `uband + lband + 1` rows in which Jacobian diagonals are written.
    Specifically `jac_packed[uband + i - j , j] = jac[i, j]`.
    The same format is used in `scipy.linalg.solve_banded` (check for an illustration).
    These parameters can be also used with `jac=None` to reduce the number of Jacobian elements estimated by finite differences.
17. `min_step` : `float`, optional

    The minimum allowed step size for 'LSODA' method.

Returns
-------
`Bunch` object with the following fields defined:
- `t` : `ndarray`, shape (n_points,)
  
  Time points.
- `y` : `ndarray`, shape (n, n_points)
  
  Values of the solution at `t`.
- `sol` : `OdeSolution` or `None`
  
  Found solution as `OdeSolution` instance; `None` if `dense_output` was set to `False`.
- `t_events` : list of `ndarray` or `None`
  
  Contains for each event type a list of arrays at which an event of that type event was detected. `None` if `events` was `None`.
- `y_events` : list of `ndarray` or `None`
  
  For each value of `t_events`, the corresponding value of the solution. None if `events` was None.
- `nfev` : `int`
  
  Number of evaluations of the right-hand side.
- `njev` : `int`
  
  Number of evaluations of the Jacobian.
- `nlu` : `int`
  
  Number of LU decompositions.
- `status` : `int`
  
  Reason for algorithm termination:
  * -1: Integration step failed.
  * 0: The solver successfully reached the end of `tspan`.
  * 1: A termination event occurred.
- `message` : `str`
  
  Human-readable description of the termination reason.
- `success` : `bool`
  
  `True` if the solver reached the interval end or a termination event occurred (`status >= 0`).

# scipy.solve_bvp

Solve a boundary-value problem for a system of ODEs.

    This function numerically solves a first order system of ODEs subject to
    two-point boundary conditions::

        dy / dx = f(x, y, p) + S * y / (x - a), a <= x <= b
        bc(y(a), y(b), p) = 0

    Here x is a 1-dimensional independent variable, y(x) is a n-dimensional
    vector-valued function and p is a k-dimensional vector of unknown
    parameters which is to be found along with y(x). For the problem to be
    determined there must be n + k boundary conditions, i.e. bc must be
    (n + k)-dimensional function.

    The last singular term in the right-hand side of the system is optional.
    It is defined by an n-by-n matrix S, such that the solution must satisfy
    S y(a) = 0. This condition will be forced during iterations, so it must not
    contradict boundary conditions. See [2]_ for the explanation how this term
    is handled when solving BVPs numerically.

    Problems in a complex domain can be solved as well. In this case y and p
    are considered to be complex, and f and bc are assumed to be complex-valued
    functions, but x stays real. Note that f and bc must be complex
    differentiable (satisfy Cauchy-Riemann equations [4]_), otherwise you
    should rewrite your problem for real and imaginary parts separately. To
    solve a problem in a complex domain, pass an initial guess for y with a
    complex data type (see below).

    Parameters
    ----------
    fun : callable
        Right-hand side of the system. The calling signature is ``fun(x, y)``,
        or ``fun(x, y, p)`` if parameters are present. All arguments are
        ndarray: ``x`` with shape (m,), ``y`` with shape (n, m), meaning that
        ``y[:, i]`` corresponds to ``x[i]``, and ``p`` with shape (k,). The
        return value must be an array with shape (n, m) and with the same
        layout as ``y``.
    bc : callable
        Function evaluating residuals of the boundary conditions. The calling
        signature is ``bc(ya, yb)``, or ``bc(ya, yb, p)`` if parameters are
        present. All arguments are ndarray: ``ya`` and ``yb`` with shape (n,),
        and ``p`` with shape (k,). The return value must be an array with
        shape (n + k,).
    x : array_like, shape (m,)
        Initial mesh. Must be a strictly increasing sequence of real numbers
        with ``x[0]=a`` and ``x[-1]=b``.
    y : array_like, shape (n, m)
        Initial guess for the function values at the mesh nodes, i-th column
        corresponds to ``x[i]``. For problems in a complex domain pass `y`
        with a complex data type (even if the initial guess is purely real).
    p : array_like with shape (k,) or None, optional
        Initial guess for the unknown parameters. If None (default), it is
        assumed that the problem doesn't depend on any parameters.
    S : array_like with shape (n, n) or None
        Matrix defining the singular term. If None (default), the problem is
        solved without the singular term.
    fun_jac : callable or None, optional
        Function computing derivatives of f with respect to y and p. The
        calling signature is ``fun_jac(x, y)``, or ``fun_jac(x, y, p)`` if
        parameters are present. The return must contain 1 or 2 elements in the
        following order:

            * df_dy : array_like with shape (n, n, m) where an element
              (i, j, q) equals to d f_i(x_q, y_q, p) / d (y_q)_j.
            * df_dp : array_like with shape (n, k, m) where an element
              (i, j, q) equals to d f_i(x_q, y_q, p) / d p_j.

        Here q numbers nodes at which x and y are defined, whereas i and j
        number vector components. If the problem is solved without unknown
        parameters df_dp should not be returned.

        If `fun_jac` is None (default), the derivatives will be estimated
        by the forward finite differences.
    bc_jac : callable or None, optional
        Function computing derivatives of bc with respect to ya, yb and p.
        The calling signature is ``bc_jac(ya, yb)``, or ``bc_jac(ya, yb, p)``
        if parameters are present. The return must contain 2 or 3 elements in
        the following order:

            * dbc_dya : array_like with shape (n, n) where an element (i, j)
              equals to d bc_i(ya, yb, p) / d ya_j.
            * dbc_dyb : array_like with shape (n, n) where an element (i, j)
              equals to d bc_i(ya, yb, p) / d yb_j.
            * dbc_dp : array_like with shape (n, k) where an element (i, j)
              equals to d bc_i(ya, yb, p) / d p_j.

        If the problem is solved without unknown parameters dbc_dp should not
        be returned.

        If `bc_jac` is None (default), the derivatives will be estimated by
        the forward finite differences.
    tol : float, optional
        Desired tolerance of the solution. If we define ``r = y' - f(x, y)``
        where y is the found solution, then the solver tries to achieve on each
        mesh interval ``norm(r / (1 + abs(f)) < tol``, where ``norm`` is
        estimated in a root mean squared sense (using a numerical quadrature
        formula). Default is 1e-3.
    max_nodes : int, optional
        Maximum allowed number of the mesh nodes. If exceeded, the algorithm
        terminates. Default is 1000.
    verbose : {0, 1, 2}, optional
        Level of algorithm's verbosity:

            * 0 (default) : work silently.
            * 1 : display a termination report.
            * 2 : display progress during iterations.
    bc_tol : float, optional
        Desired absolute tolerance for the boundary condition residuals: `bc` 
        value should satisfy ``abs(bc) < bc_tol`` component-wise. 
        Equals to `tol` by default. Up to 10 iterations are allowed to achieve this
        tolerance.

    Returns
    -------
    Bunch object with the following fields defined:
    sol : PPoly
        Found solution for y as `scipy.interpolate.PPoly` instance, a C1
        continuous cubic spline.
    p : ndarray or None, shape (k,)
        Found parameters. None, if the parameters were not present in the
        problem.
    x : ndarray, shape (m,)
        Nodes of the final mesh.
    y : ndarray, shape (n, m)
        Solution values at the mesh nodes.
    yp : ndarray, shape (n, m)
        Solution derivatives at the mesh nodes.
    rms_residuals : ndarray, shape (m - 1,)
        RMS values of the relative residuals over each mesh interval (see the
        description of `tol` parameter).
    niter : int
        Number of completed iterations.
    status : int
        Reason for algorithm termination:

            * 0: The algorithm converged to the desired accuracy.
            * 1: The maximum number of mesh nodes is exceeded.
            * 2: A singular Jacobian encountered when solving the collocation
              system.

    message : string
        Verbal description of the termination reason.
    success : bool
        True if the algorithm converged to the desired accuracy (``status=0``).
"""
